In [1]:
import qsharp

qsharp.init(target_profile=qsharp.TargetProfile.Base)

Q# initialized with configuration: {'targetProfile': 'base', 'languageFeatures': None, 'manifest': None}

In [2]:
%%qsharp

operation MyH(q: Qubit) : Unit {
  Rz(3.141592653589793 / 2.0, q);
  SX(q);
  Rz(3.141592653589793 / 2.0, q);
}

operation MyCX(control: Qubit, target: Qubit) : Unit {
  MyH(target);
  CZ(control, target);
  MyH(target);
}

operation Main() : Result[] {
  use q = Qubit[2];

  MyH(q[0]);
  MyCX(q[0], q[1]);

  return MResetEachZ(q);
}

In [3]:
from qsharp_widgets import Circuit

Circuit(qsharp.circuit(qsharp.code.Main))

Circuit(circuit_json='{"qubits":[{"id":0,"numResults":1},{"id":1,"numResults":1}],"componentGrid":[{"component…

In [4]:
qsharp.run(qsharp.code.Main, shots=10)

[[Zero, Zero],
 [Zero, Zero],
 [Zero, Zero],
 [One, One],
 [One, One],
 [One, One],
 [One, One],
 [One, One],
 [One, One],
 [One, One]]

In [5]:
from nasm import Simulator, create_default_noise_model, amplitude_damping_kraus

qir = qsharp.compile(qsharp.code.Main)

noise = create_default_noise_model()
sim = Simulator(noise_model=noise)
results = sim.run(qir, shots=10)
print("\n".join(results))


00
11
11
00
11
11
11
00
11
00


In [13]:
noise.add_kraus_operator("amp_damp_015", amplitude_damping_kraus(0.15))
noise.update_gate_noise("sx", "amp_damp_015")
noise.update_gate_noise("rz", "amp_damp_015")

results = sim.run(qir, shots=10)
print("\n".join(results))


00
00
10
01
00
11
10
00
11
00


In [7]:
# TODO: Qubit loss